In [ ]:
import gym
import py4j
from py4j.java_gateway import JavaGateway
import random
import time
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import math
from scipy import stats
import seaborn as sns
from time import process_time

import matplotlib.pyplot as plt

In [ ]:
class HHEnv(gym.Env):
    def __init__(self,problem_domain = 'TSP', time_limit=600,lubylength = 200000):
        hyflex = JavaGateway().jvm
        self.action_space = None
        self.problem_domain = problem_domain
        if problem_domain == 'SAT':   
            self.problem = hyflex.SAT.SAT(1234)
        elif problem_domain =='BP':
            self.problem = hyflex.BinPacking.BinPacking(1234) 
        elif problem_domain == 'TSP':
            self.problem = hyflex.travelingSalesmanProblem.TSP(1234)
        
        self.last_h = 9
        self.tail_length = 10
        
        low = np.array(
            [   -1,
                -1
            ],
            dtype=np.float32,
        )
        high = np.array(
            [   self.last_h,
                self.tail_length
            ],
            dtype=np.float32,
        )
        self.observation_space = Box(low, high)
        self.state = None
        self.done = False
        self.solution = None
        self.h_chain = None
        self.luby = None
        self.episode = 0
        self.lubylength = lubylength
        self.origin_sol = None
        self.time_limit = time_limit
        self.current_time = None
        self.overall_time = 0
        self.exceed = False
        self.totalDelta = 0
        self.sidewaysMoveFound = False
        self.costCurrent = None
        
        def luby(length = 200000):
            L = [-1]*length
            for i in range (1,length+1):
        #         print("current index: ",i)
                left = math.log2(i + 1)
                right = int(left)
                k = right
                if left == right:
                    L[i-1] = 2 **(k-1)
                else:
                    k+=1
                    L[i-1] = L[i-1 - 2**(k-1) + 1]
            return L
        
        self.luby = luby(length=self.lubylength)
        
    def step(self, action):
        
        # setting time 
        t1 = time.time()
        
        # last heuristic applied
        last_heu = self.h_chain[self.iter_num-1]
        
        # update hyper-heuristic chain
        self.h_chain[self.iter_num] = int(action)
        
        self.iter_num += 1  
        
        # number of heuristic left if bigger than max tail length it equal max tail length
        num_heur_left = len(self.h_chain) - self.iter_num
        
        num_heur_left = min(self.tail_length, num_heur_left)
        
        ori = self.origin_sol
        
        self.origin_sol = self.problem.applyHeuristic(int(action), 1, 1)
        delta = self.origin_sol - ori
        
        t2 = time.time()
    
        self.current_time += t2-t1
        self.overall_time += t2-t1
        self.totalDelta+=delta

        end = True if (len(self.h_chain)) == self.iter_num else False
        
        if end:
            self.done = True

        if self.totalDelta <0:
            self.current_time = max(1,self.current_time)
            reward = (- self.totalDelta)/(self.current_time)
        else:
            reward = 0
    
            
        self.state = (last_heu, num_heur_left)
        
        info = {}
        # Return step information
        self.solution = self.problem.getBestSolutionValue()
        if self.overall_time > self.time_limit:
            self.exceed == True
        if self.totalDelta < 0:
            self.problem.copySolution(1,0)
            self.done = True
        elif self.totalDelta == 0:
            self.problem.copySolution(1,2)
            self.sidewaysMoveFound = True
        elif end and self.sidewaysMoveFound and self.totalDelta >= 0:
            self.problem.copySolution(2,0)
            
        return np.array(self.state, dtype=np.float32), reward, self.done, info

    def render(self):
        # Implement viz
        pass
        
    
    def initialize(self,instance):
        self.problem.loadInstance(instance)
        self.problem.setMemorySize(4)
        self.problem.initialiseSolution(0)
        self.problem.copySolution(0,1)
        self.problem.copySolution(0,3)
        self.origin_sol = self.problem.getFunctionValue(0)
        self.action_space = Discrete(self.problem.getNumberOfHeuristics())
        self.iter_num = 0
        self.current_time = 0
        self.h_chain = [-1 for i in range(self.luby[self.episode])]
        self.episode += 1
        self.state = (-1, min(10,len(self.h_chain)))
        self.sidewaysMoveFound = False
        self.done = False
        return np.array(self.state, dtype=np.float32)
    
    def reset(self):
        self.origin_sol = self.problem.getBestSolutionValue()
        self.problem.copySolution(0,1)
        self.sidewaysMoveFound = False
        self.iter_num = 0
        self.current_time = 0
        self.totalDelta = 0
        self.h_chain = [-1 for i in range(self.luby[self.episode])]
        self.episode += 1
        self.state = (-1, min(10,len(self.h_chain)))
        self.done = False
        return np.array(self.state, dtype=np.float32)
    
    def start_from_previous(self):
        self.problem.copySolution(3, 1)
    
    def store_best(self):
        self.problem.copySolution(0, 3)

In [ ]:
class HHagent():
    def __init__(self, problem_domain = 'SAT',
                 instance = 3, time = 100, min_lr = None, 
                 min_epsilon = None, discount = None, decay = None,  unimprovement_time = None,
                 is_backtrack = False, random_store = False, is_restart = False, res_q = False):
        
        self.problem_domain = problem_domain
        self.env = HHEnv(self.problem_domain)
        self.instance = instance
        self.env.initialize(self.instance)        
        self.buckets = (self.env.action_space.n + 1, self.env.tail_length + 1)
        
        self.time = time
        self.min_lr = 0.3
        self.min_epsilon = 0.1
        self.discount = 0.5
        self.decay = 25
        self.unimprovement_time = 20
        self.is_backtrack = is_backtrack
        self.random_store = random_store
        self.is_restart = is_restart
        self.res_q = res_q
        self.Q_table = np.zeros(self.buckets + (self.env.action_space.n,))
        self.funV = []
        self.functionValue = []
        self.Bests = []
        
        
    def discretize_state(self, obs):
        discretized = list()
        for i in range(len(obs)):
            discretized.append(int(obs[i]))

        return tuple(discretized)

    def choose_action(self, state):
        if (np.random.random() < self.epsilon):
            return self.env.action_space.sample() 
        else:
            return np.argmax(self.Q_table[state])

    def update_q(self, state, action, reward, new_state):
        self.Q_table[state][action] += (self.learning_rate * 
                                        (reward 
                                         + self.discount * np.max(self.Q_table[new_state]) 
                                         - self.Q_table[state][action]))
        

    def get_epsilon(self, t):
        
        return max(self.min_epsilon, min(1., 1. - math.log10((t + 1) / self.decay)))

    def get_learning_rate(self, t):
        return max(self.min_lr, min(1., 1. - math.log10((t + 1) / self.decay)))
    
    
    def train(self):
        e = -1
        overall_time = 0
        start_time = time.time()
        best = np.inf
        best_time = start_time
        best_of_all_time = np.inf
        
        while self.time > overall_time:
            e += 1
            end_time = time.time()
            overall_time = end_time - start_time
            
            # reset or intiailize the state
            if end_time - best_time > self.unimprovement_time:
                if self.is_restart:
                    print("+++++++++UPDATE_RESTART+++++++++++++++++")
                    self.Bests.append(self.env.problem.getBestSolutionValue())
                    best = np.inf
                    self.env = HHEnv(self.problem_domain)
                    current_state = self.discretize_state(self.env.initialize(self.instance))
                    if self.res_q:
                        self.Q_table = np.zeros(self.buckets + (self.env.action_space.n,))
                if self.is_backtrack:
                    print("+++++++++Go to Random Best+++++++++++++++++")
                    self.Bests.append(self.env.problem.getBestSolutionValue())
                    self.env.start_from_previous()
                    best = np.inf
                    current_state = self.discretize_state(self.env.reset())
                else:
                    self.Bests.append(self.env.problem.getBestSolutionValue())
                    current_state = self.discretize_state(self.env.reset())
            else:
                self.Bests.append(self.env.problem.getBestSolutionValue())
                current_state = self.discretize_state(self.env.reset())

            self.learning_rate = self.get_learning_rate(e)
            self.epsilon = self.get_epsilon(e)
            done = False
            
            # Looping for each step
            while not done:
                # Choose A from S
                action = self.choose_action(current_state)
                
                # Take action
                obs, reward, done, _ = self.env.step(action)
                new_state = self.discretize_state(obs)
                # Update Q(S,A)
                self.update_q(current_state, action, reward, new_state)
                current_state = new_state
                
                # We break out of the loop when done is False which is
                # a terminal state.
            if e % 500 == 0:
                print("Current Funciton Value: {}".format(self.env.problem.getFunctionValue(0)))
            
            if self.env.problem.getFunctionValue(0) < best:
    
                best = self.env.problem.getFunctionValue(0)
                best_time = end_time
                if self.is_backtrack:
                    if self.random_store:
                        random_store = random.uniform(0, 1)
                        if random_store < 0.3:
                            self.env.store_best()
                    self.env.store_best()
            
            if best_of_all_time > self.env.problem.getBestSolutionValue():
                best_of_all_time = self.env.problem.getBestSolutionValue()
                
            self.funV.append(self.env.problem.getFunctionValue(0))
        print('Finished training! Episode is {}'.format(e))
        print("Recorded best solutnion is {}".format(best_of_all_time))
    
        return self.env.problem.getBestSolutionValue(), min(self.Bests)
    
    def plot_learning(self):
        """
        Plots the number of steps at each episode and prints the
        amount of times that an episode was successfully completed.
        """
        sns.lineplot(x = range(len(self.funV)), y = self.funV)
        
    def save(self):
        np.save('Qtable',self.Q_table)

In [ ]:
def load_q_learning():
     agent = HHagent(problem_domain='SAT',instance = 3,time = 50)
     a, b = agent.train()
     agent.plot_learning()
     
     return a, b

In [ ]:
def run_all():
    best_org = []
    best_restart = []
    best_restart_q = []
    best_backtrack = []
    best_backtrack_random = []
    
    plt.figure()
    plt.title('org')
    for i in range(2):
#         print(f'######{i}for org')
        agent = HHagent(problem_domain='SAT',instance = 3,time = 600)
        print("Training orginal")
        best = agent.train()
        best_org.append(best)
        agent.plot_learning()

    
    plt.figure()
    plt.title('Restart')
    for i in range(2):
#         print(f'######{i}for restart')
        agent = HHagent(problem_domain = 'SAT', instance = 3, time = 600, is_restart = True)
        print('Training Restart')
        best = agent.train()
        best_restart.append(best)
        agent.plot_learning()
        
    plt.figure()
    plt.title('Restart with q')
    for i in range(2):
#         print(f'######{i}for Restart with q')
        agent = HHagent(problem_domain = 'SAT', instance = 3, time = 600, is_restart = True, res_q = True)
        print('Training Restart and Q')
        best = agent.train()
        best_restart_q.append(best)
        agent.plot_learning()
    
    plt.figure()
    plt.title('Backtrack')
    for i in range(2):
        (f'######{i}for Restart with q')
        agent = HHagent(problem_domain = 'SAT', instance = 3, time = 600, is_backtrack = True)
        print("Training Backtrack")
        best = agent.train()
        best_backtrack.append(best)
        agent.plot_learning() 
    
    plt.figure()
    plt.title('Backtrack with random')
    for i in range(2):
        agent = HHagent(problem_domain = 'SAT', instance = 3, time = 600, is_backtrack = True, random_store = True)
        print("Training BackTrack with Random")
        best = agent.train()
        best_backtrack_random.append(best)
        agent.plot_learning() 
    
    
    return best_org, best_restart, best_restart_q, best_backtrack, best_backtrack_random

In [ ]:
best_org, best_restart, best_restart_q, best_backtrack, best_backtrack_random = run_all()

In [ ]:
print("best_org", best_org)
print(np.mean(best_org))
print("best_restart", best_restart)
print(np.mean(best_restart))
print("best_restart_q", best_restart_q)
print(np.mean(best_restart_q))
print("best_backtrack", best_backtrack)
print(np.mean(best_backtrack))
print("best_backtrack_random", best_backtrack_random)
print(np.mean(best_backtrack_random))

In [ ]:
def run_all():
    best_org = []
    best_restart = []
    best_restart_q = []
    best_backtrack = []
    best_backtrack_random = []
    
    plt.figure()
    plt.title('org')
    for i in range(4):
#         print(f'######{i}for org')
        agent = HHagent(problem_domain='TSP',instance = 0,time = 600)
        print("Training orginal")
        best = agent.train()
        best_org.append(best)
        agent.plot_learning()

    
    plt.figure()
    plt.title('Restart')
    for i in range(4):
#         print(f'######{i}for restart')
        agent = HHagent(problem_domain = 'TSP', instance = 0, time = 600, is_restart = True)
        print('Training Restart')
        best = agent.train()
        best_restart.append(best)
        agent.plot_learning()
        
    plt.figure()
    plt.title('Restart with q')
    for i in range(4):
#         print(f'######{i}for Restart with q')
        agent = HHagent(problem_domain = 'TSP', instance = 0, time = 600, is_restart = True, res_q = True)
        print('Training Restart and Q')
        best = agent.train()
        best_restart_q.append(best)
        agent.plot_learning()
    
    plt.figure()
    plt.title('Backtrack')
    for i in range(4):
        (f'######{i}for Restart with q')
        agent = HHagent(problem_domain = 'TSP', instance = 0, time = 600, is_backtrack = True)
        print("Training Backtrack")
        best = agent.train()
        best_backtrack.append(best)
        agent.plot_learning() 
    
    plt.figure()
    plt.title('Backtrack with random')
    for i in range(4):
        agent = HHagent(problem_domain = 'TSP', instance = 0, time = 600, is_backtrack = True, random_store = True)
        print("Training BackTrack with Random")
        best = agent.train()
        best_backtrack_random.append(best)
        agent.plot_learning() 
    
    
    return best_org, best_restart, best_restart_q, best_backtrack, best_backtrack_random


best_org, best_restart, best_restart_q, best_backtrack, best_backtrack_random = run_all()
print("best_org", best_org)
print(np.mean(best_org))
print("best_restart", best_restart)
print(np.mean(best_restart))
print("best_restart_q", best_restart_q)
print(np.mean(best_restart_q))
print("best_backtrack", best_backtrack)
print(np.mean(best_backtrack))
print("best_backtrack_random", best_backtrack_random)
print(np.mean(best_backtrack_random))

In [ ]:
def run_all():
    best_org = []
    best_restart = []
    best_restart_q = []
    best_backtrack = []
    best_backtrack_random = []
    
    plt.figure()
    plt.title('org')
    for i in range(2):
#         print(f'######{i}for org')
        agent = HHagent(problem_domain='TSP',instance = 3,time = 600)
        print("Training orginal")
        best = agent.train()
        best_org.append(best)
        agent.plot_learning()

    
    plt.figure()
    plt.title('Restart')
    for i in range(2):
#         print(f'######{i}for restart')
        agent = HHagent(problem_domain = 'TSP', instance = 3, time = 600, is_restart = True)
        print('Training Restart')
        best = agent.train()
        best_restart.append(best)
        agent.plot_learning()
        
    plt.figure()
    plt.title('Restart with q')
    for i in range(2):
#         print(f'######{i}for Restart with q')
        agent = HHagent(problem_domain = 'TSP', instance = 3, time = 600, is_restart = True, res_q = True)
        print('Training Restart and Q')
        best = agent.train()
        best_restart_q.append(best)
        agent.plot_learning()
    
    plt.figure()
    plt.title('Backtrack')
    for i in range(2):
        (f'######{i}for Restart with q')
        agent = HHagent(problem_domain = 'TSP', instance = 3, time = 600, is_backtrack = True)
        print("Training Backtrack")
        best = agent.train()
        best_backtrack.append(best)
        agent.plot_learning() 
    
    plt.figure()
    plt.title('Backtrack with random')
    for i in range(2):
        agent = HHagent(problem_domain = 'TSP', instance = 3, time = 600, is_backtrack = True, random_store = True)
        print("Training BackTrack with Random")
        best = agent.train()
        best_backtrack_random.append(best)
        agent.plot_learning() 
    
    
    return best_org, best_restart, best_restart_q, best_backtrack, best_backtrack_random

best_org, best_restart, best_restart_q, best_backtrack, best_backtrack_random = run_all()
print("best_org", best_org)
print(np.mean(best_org))
print("best_restart", best_restart)
print(np.mean(best_restart))
print("best_restart_q", best_restart_q)
print(np.mean(best_restart_q))
print("best_backtrack", best_backtrack)
print(np.mean(best_backtrack))
print("best_backtrack_random", best_backtrack_random)
print(np.mean(best_backtrack_random))